In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# Paths 
BASE_DIR = r"C:\Users\Varun\OneDrive\Pictures\Desktop\waste_classification_project\application"
SPLIT_DIR = os.path.join(BASE_DIR, "data", "split")

In [3]:
train_dir = os.path.join(SPLIT_DIR, "train")
val_dir   = os.path.join(SPLIT_DIR, "val")
test_dir  = os.path.join(SPLIT_DIR, "test")

In [4]:
#  Parameters 
IMG_SIZE = (224, 224)  # ResNet expects 224x224
BATCH_SIZE = 32
EPOCHS = 10

In [5]:
#  Data Generators 
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # important for ResNet
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)

In [6]:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [7]:
if not os.path.exists(train_dir):
    print(f"Train directory not found: {train_dir}")
else:
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        color_mode='rgb'
    )
    print(f"Found {train_generator.samples} training images belonging to {train_generator.num_classes} classes.")   

Found 16598 images belonging to 10 classes.
Found 16598 training images belonging to 10 classes.


In [8]:
if not os.path.exists(val_dir):
    print(f"Validation directory not found: {val_dir}")
else:
    val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        color_mode='rgb'
    )
    print(f"Found {val_generator.samples} validation images belonging to {val_generator.num_classes} classes.")

Found 3553 images belonging to 10 classes.
Found 3553 validation images belonging to 10 classes.


In [9]:
if not os.path.exists(test_dir):
    print(f"Test directory not found: {test_dir}")
else:
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        color_mode='rgb'
    )
    print(f"Found {test_generator.samples} test images belonging to {test_generator.num_classes} classes.")

Found 3567 images belonging to 10 classes.
Found 3567 test images belonging to 10 classes.


In [10]:
num_classes = len(train_generator.class_indices)
print("Class indices:", train_generator.class_indices)

Class indices: {'Electronic waste': 0, 'cardboard waste': 1, 'clothe waste': 2, 'glass waste': 3, 'metal waste': 4, 'organic waste': 5, 'paper waste': 6, 'plastic waste': 7, 'shoes waste': 8, 'trash': 9}


In [11]:
# Check shape
batch_x, batch_y = next(train_generator)
print("Batch image shape:", batch_x.shape)  # (batch_size, 224, 224, 3)

Batch image shape: (32, 224, 224, 3)


In [12]:
#  Build ResNet model 
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # freeze base layers

In [13]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,608,202 (90.06 MB)

 Trainable params: 20,490 (80.04 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [14]:
#  Callbacks 
os.makedirs(os.path.join(BASE_DIR, "models"), exist_ok=True)
checkpoint_path = os.path.join(BASE_DIR, "models", "waste_model_resnet.keras")

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)
]

In [15]:
#  Train head 
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks
)

c:\Users\Varun\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 1875s 4s/step - accuracy: 0.7816 - loss: 0.6772 - val_accuracy: 0.9066 - val_loss: 0.2957 - learning_rate: 0.0010
Epoch 2/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 1508s 3s/step - accuracy: 0.8662 - loss: 0.4157 - val_accuracy: 0.9181 - val_loss: 0.2609 - learning_rate: 0.0010
Epoch 3/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 1522s 3s/step - accuracy: 0.8761 - loss: 0.3869 - val_accuracy: 0.9142 - val_loss: 0.2572 - learning_rate: 0.0010
Epoch 4/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 1500s 3s/step - accuracy: 0.8802 - loss: 0.3636 - val_accuracy: 0.9294 - val_loss: 0.2382 - learning_rate: 0.0010
Epoch 5/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 1542s 3s/step - accuracy: 0.8858 - loss: 0.3558 - val_accuracy: 0.9274 - val_loss: 0.2238 - learning_rate: 0.0010
Epoch 6/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 1454s 3s/step - accuracy: 0.8881 - loss: 0.3493 - val_accuracy: 0.9341 - val_loss: 0.2158 - learning_rate: 0.0010
Epoch 7/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 1707s 3s/step - accuracy: 0.8864 - l

In [16]:
#  Evaluate before fine-tuning
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc*100:.2f}%")

112/112 ━━━━━━━━━━━━━━━━━━━━ 263s 2s/step - accuracy: 0.9324 - loss: 0.2148
Test Accuracy: 93.24%


In [17]:
#  Fine-tune top ResNet layers 
base_model.trainable = True
for layer in base_model.layers[:-50]:  # freeze lower layers
    layer.trainable = False

In [18]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
history_fine = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=callbacks
)

Epoch 1/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 2258s 4s/step - accuracy: 0.8989 - loss: 0.3268 - val_accuracy: 0.9443 - val_loss: 0.2210 - learning_rate: 1.0000e-04
Epoch 2/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 2418s 5s/step - accuracy: 0.9396 - loss: 0.1840 - val_accuracy: 0.9482 - val_loss: 0.1721 - learning_rate: 1.0000e-04
Epoch 3/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 2549s 5s/step - accuracy: 0.9543 - loss: 0.1381 - val_accuracy: 0.9575 - val_loss: 0.1736 - learning_rate: 1.0000e-04
Epoch 4/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 2473s 5s/step - accuracy: 0.9637 - loss: 0.1126 - val_accuracy: 0.9612 - val_loss: 0.1482 - learning_rate: 1.0000e-04
Epoch 5/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 2537s 5s/step - accuracy: 0.9723 - loss: 0.0905 - val_accuracy: 0.9595 - val_loss: 0.1533 - learning_rate: 1.0000e-04
Epoch 6/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 2297s 4s/step - accuracy: 0.9718 - loss: 0.0852 - val_accuracy: 0.9614 - val_loss: 0.1494 - learning_rate: 1.0000e-04
Epoch 7/10
519/519 ━━━━━━━━━━━━━━━━━━━━ 2274s 4s/ste

In [20]:
# Final evaluation
test_loss, test_acc = model.evaluate(test_generator) 
print(f"Test Accuracy after fine-tuning: {test_acc*100:.2f}%")

112/112 ━━━━━━━━━━━━━━━━━━━━ 254s 2s/step - accuracy: 0.9689 - loss: 0.1715
Test Accuracy after fine-tuning: 96.89%


In [21]:
# Save the full fine-tuned ResNet model
final_model_path = os.path.join(BASE_DIR, "models", "final_waste_model_resnet.keras")
model.save(final_model_path)
print(f"Model saved successfully ✅ at {final_model_path}")

Model saved successfully ✅ at C:\Users\Varun\OneDrive\Pictures\Desktop\waste_classification_project\application\models\final_waste_model_resnet.keras
